In [79]:
"""
Import Statements: From PyTorch RL Tutorial
"""
import gym
import math
import random
import numpy as np
from itertools import count
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import nle
from collections import namedtuple, deque

# if gpu is to be used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

env = gym.make('NetHackChallenge-v0', savedir=None)

In [94]:
"""
This model stores each individual experience replay.
"""
class Experience():
    def __init__(self, state, action, next_state, reward):
        self.state - state
        self.action = action
        self.next_state = next_state
        self.reward = reward

In [81]:
StateTransition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))

In [95]:
"""
This model stores replays of memories as a whole. It also holds the full set, and manages removing old memories.
"""
class ReplayMemory():
    def __init__(self, max_memory):
        self.memory = []
        self.max_memory = max_memory
    
    def push(self, newExp):
        if len(self.memory) >= self.max_memory:
            self.memory.pop(0)
        self.memory.append(newExp)
    
    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)
    
    def __len__(self):
        return len(self.memory)

In [96]:
"""
DQN implementation, with convolutional layers, max pooling implemented.
"""
class DQNetwork(nn.Module):
    def __init__(self):
        super(DQNetwork, self).__init__()
        self.joe = nn.Sequential(
            nn.Conv2d(4, 16, kernel_size=3, padding='same'),
            nn.BatchNorm2d(16),
            nn.ReLU(), # 21 * 79 * 16
            nn.MaxPool2d(3, padding=(0,1)), # 7 * 27 * 16
            nn.Conv2d(16, 64, kernel_size=3, padding='same'),# 7 * 27 * 64
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(3, padding=(1,0)), # 3 * 9 * 64
            nn.Flatten(),
            nn.Linear(3*9*64, 3*64),
            nn.ReLU(),
            nn.Linear(3*64, 113),
        )
        
    def forward(self, x):
        logits = self.joe(x.float())
        return logits

In [84]:
x = np.load('l.txt.npy')
x = torch.Tensor(x)[None,:,:,:]
print(x.shape)

torch.Size([1, 4, 21, 79])


In [85]:
bink = DQNetwork()
bink.forward(x)

tensor([[ 0.5820, -0.0145, -0.1973, -0.1992,  0.2478, -0.0486, -0.3093, -0.5386,
         -0.3617,  0.3553, -0.1044,  0.0896,  0.3319,  0.0839,  0.3118, -0.1289,
          0.6876, -0.3061,  0.2591, -0.1896, -0.2452, -0.3310,  0.4667, -0.0732,
          0.0860,  0.2486,  0.4494, -0.1109, -0.2731, -0.3920, -0.3173,  0.1012,
         -0.3699,  0.3519, -0.5330,  0.1259,  0.2164,  0.1370,  0.0064,  0.1190,
         -0.4528,  0.3458,  0.1344, -0.0987,  0.1707, -0.2028, -0.1253,  0.2144,
         -0.1029,  0.3621, -0.2257,  0.3409,  0.3094,  0.2402, -0.4096,  0.4916,
          0.4554,  0.2320, -0.2696,  0.0756,  0.1477, -0.0558, -0.1738,  0.0380,
          0.3713, -0.7179, -0.0909, -0.0165, -0.0039,  0.2123, -0.2760, -0.0745,
          0.1062, -0.7338, -0.1024, -0.3925,  0.4214, -0.5662,  0.0836, -0.1241,
         -0.2492, -0.0139,  0.1420,  0.1919,  0.1639, -0.0694,  0.1629,  0.0275,
          0.1929, -0.1990,  0.0890,  0.3104,  0.0475, -0.4032,  0.3717,  0.0449,
          0.1330,  0.3433, -

In [97]:
BATCH_SIZE = 128
GAMMA = 0.999
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 200
TARGET_UPDATE = 10
steps_done = 0
episode_durations = []
n_actions = env.action_space.n
# All of the batches

In [87]:
policy_net = DQNetwork().to(device)
target_net = DQNetwork().to(device)
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()
optimizer = optim.RMSprop(policy_net.parameters())
memory = ReplayMemory(10000)

In [88]:
def select_action(state):
    global steps_done
    sample = random.random()
    eps_threshold = EPS_END + (EPS_START - EPS_END) * \
        math.exp(-1. * steps_done / EPS_DECAY)
    steps_done += 1
    if sample > eps_threshold:
        with torch.no_grad():
            # t.max(1) will return largest column value of each row.
            # second column on max result is index of where max element was
            # found, so we pick action with the larger expected reward.
            return policy_net(state).max(1)[1].view(1, 1)
    else:
        return torch.tensor([[random.randrange(n_actions)]], device=device, dtype=torch.long)

In [91]:
def optimize_model(iter_num):
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)
    # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
    # detailed explanation). This converts batch-array of Transitions
    # to Transition of batch-arrays.
    batch = StateTransition(*zip(*transitions))

    # Compute a mask of non-final states and concatenate the batch elements
    # (a final state would've been the one after which simulation ended)
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)), device=device, dtype=torch.bool)
    non_final_next_states = torch.cat([s for s in batch.next_state
                                                if s is not None])
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)

    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken. These are the actions which would've been taken
    # for each batch state according to policy_net
    state_action_values = policy_net(state_batch).gather(1, action_batch)

    # Compute V(s_{t+1}) for all next states.
    # Expected values of actions for non_final_next_states are computed based
    # on the "older" target_net; selecting their best reward with max(1)[0].
    # This is merged based on the mask, such that we'll have either the expected
    # state value or 0 in case the state was final.
    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    next_state_values[non_final_mask] = target_net(non_final_next_states).max(1)[0].detach()
    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    # Compute Huber loss
    criterion = nn.SmoothL1Loss()
    loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))
    if iter_num % 100 == 0:
        print('Iteration number ' + str(iter_num) + ' loss: ' + str(loss.item()))
    
    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    for param in policy_net.parameters():
        param.grad.data.clamp_(-1, 1)
    optimizer.step()

In [92]:
def concat_state(state):
    state = np.stack((state['glyphs'], state['chars'], state['colors'], state['specials']))
    state = torch.tensor(np.expand_dims(state, axis=0), device=device)
    return state

# EPISODES
num_episodes = 20
for i_episode in range(num_episodes):
    print('Episode Number: ' + str(i_episode))
    # Initialize the environment and state
    obs = env.reset()
    state = concat_state(obs)
    for t in count():
        # Select and perform an action
        action = select_action(state)
        obs, reward, done, _ = env.step(action.item())
        reward = torch.tensor([reward], device=device)
        
        # Keep track of current state and previous states 
        last_state = state
        state = concat_state(obs)

        # Store the transition in memory
        memory.push(StateTransition(last_state, action, state, reward))

        # Perform one step of the optimization (on the policy network)
        optimize_model(t)
        if done:
            episode_durations.append(t + 1)
            break
    # Update the target network, copying all weights and biases in DQN
    if i_episode % TARGET_UPDATE == 0:
        target_net.load_state_dict(policy_net.state_dict())
print('Complete')
env.render()
env.close()
plt.ioff()
plt.show()

Episode Number: 0
Iteration number 0 loss: 6.063097953796387
Iteration number 100 loss: 5.312196731567383
Iteration number 200 loss: 5.266094207763672
Iteration number 300 loss: 4.826352119445801
Iteration number 400 loss: 5.324923515319824
Iteration number 500 loss: 5.1598029136657715
Iteration number 600 loss: 4.959875583648682
Iteration number 700 loss: 5.1431427001953125
Iteration number 800 loss: 5.475369930267334
Iteration number 900 loss: 5.263668537139893
Iteration number 1000 loss: 5.570958614349365
Iteration number 1100 loss: 4.899526596069336
Iteration number 1200 loss: 4.876068592071533
Iteration number 1300 loss: 4.815316200256348
Iteration number 1400 loss: 4.18011474609375
Iteration number 1500 loss: 5.1479597091674805
Iteration number 1600 loss: 5.231861114501953
Iteration number 1700 loss: 5.413294792175293
Iteration number 1800 loss: 5.127189636230469
Iteration number 1900 loss: 4.941763877868652
Iteration number 2000 loss: 4.384138107299805
Iteration number 2100 loss

In [93]:
max(episode_durations)

106754

In [98]:
rewards = [x.reward for x in memory.memory]
max(rewards)

tensor([0.], device='cuda:0')